In [1]:
import os
import pandas as pd
import torch
from torch_geometric.loader import DataLoader
import mlflow
from datetime import datetime

from src.modules.graph_nn.dataset import TransactionDataset
from src.modules.graph_nn.model import TransactionGNN
from src.modules.graph_nn.train import train_epoch, evaluate_epoch


class Constants:
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    BATCH_SIZE = 32
    HIDDEN_DIM = 256
    NUM_LAYERS = 5
    NUM_HEADS = 4
    SHUFFLE = False
    TRAIN_SPLIT = 0.8

    TRANSACTIONS_PATH = "/Users/alfa/Documents/diplom/graphnn-recommendation-system/data/processed_transactions_train.csv"

In [2]:
# Загружаем транзакции и создаем torch dataset
transactions = pd.read_parquet(Constants.TRANSACTIONS_PATH)
graph_dataset = TransactionDataset(transactions)

# Разделяем на train и test
graph_train_size = int(Constants.TRAIN_SPLIT * len(graph_dataset))
graph_train_dataset = torch.utils.data.Subset(graph_dataset, range(graph_train_size))
graph_test_dataset = torch.utils.data.Subset(
    graph_dataset, range(graph_train_size, len(graph_dataset))
)

# Создаем загрузчики данных
graph_train_loader = DataLoader(
    graph_train_dataset, batch_size=Constants.BATCH_SIZE, shuffle=Constants.SHUFFLE
)
graph_test_loader = DataLoader(
    graph_test_dataset, batch_size=Constants.BATCH_SIZE, shuffle=Constants.SHUFFLE
)

# Инициализируем модель
graph_model = TransactionGNN(
    num_items=graph_dataset.num_items,
    hidden_dim=Constants.HIDDEN_DIM,
    num_layers=Constants.NUM_LAYERS,
    num_heads=Constants.NUM_HEADS,
).to(Constants.DEVICE)
graph_optimizer = torch.optim.Adam(graph_model.parameters(), lr=0.001)

# Обучение и оценка
for epoch in range(5):
    train_loss = train_epoch(
        model=graph_model,
        loader=graph_train_loader,
        optimizer=graph_optimizer,
        device=Constants.DEVICE,
    )
    test_acc, test_loss = evaluate_epoch(
        model=graph_model,
        loader=graph_test_loader,
        device=Constants.DEVICE,
    )
    print(
        f"Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Test Acc: {test_acc:.4f}, Test Loss: {test_loss:.4f}"
    )

Evaluate Epoch:: 100%|██████████| 35/35 [00:00<00:00, 266.24it/s]


Epoch: 000, Train Loss: 4.4518, Test Acc: 0.0275, Test Loss: 4.5033


Evaluate Epoch:: 100%|██████████| 35/35 [00:00<00:00, 288.49it/s]


Epoch: 001, Train Loss: 4.1285, Test Acc: 0.0009, Test Loss: 4.9181


Evaluate Epoch:: 100%|██████████| 35/35 [00:00<00:00, 270.08it/s]


Epoch: 002, Train Loss: 4.0917, Test Acc: 0.0055, Test Loss: 4.8772


Evaluate Epoch:: 100%|██████████| 35/35 [00:00<00:00, 236.58it/s]


Epoch: 003, Train Loss: 4.1012, Test Acc: 0.0183, Test Loss: 4.4578


Evaluate Epoch:: 100%|██████████| 35/35 [00:00<00:00, 301.42it/s]

Epoch: 004, Train Loss: 4.0227, Test Acc: 0.0046, Test Loss: 4.7484


# Логирование артефактов моделей в MLFlow

In [3]:
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"

In [4]:
def save_model_to_mlflow():
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment("graph-nn-model")

    with mlflow.start_run(run_name=f"model_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
        # Сохраняем модель PyTorch
        mlflow.pytorch.log_model(graph_model, "model")

        # Сохраняем энкодер товаров
        import joblib

        joblib.dump(graph_dataset.item_encoder, "item_encoder.pkl")
        mlflow.log_artifact("item_encoder.pkl", "encoders")

        # Логируем параметры модели
        mlflow.log_params(
            {
                "num_items": graph_dataset.num_items,
                "hidden_dim": Constants.HIDDEN_DIM,
                "num_layers": Constants.NUM_LAYERS,
                "num_heads": Constants.NUM_HEADS,
            }
        )


save_model_to_mlflow()

2025/04/18 19:33:39 INFO mlflow.tracking.fluent: Experiment with name 'graph-nn-model' does not exist. Creating a new experiment.
2025/04/18 19:33:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run model_20250418_193339 at: http://localhost:5000/#/experiments/1/runs/a69b307154ae4f2fb88e140355420935
🧪 View experiment at: http://localhost:5000/#/experiments/1
